**N5GEH-Serv datamodeling workshop**
==========================================


A data model is an abstraction of a model that organizes elements of data and gives an standardization on properties and relationships. Furthermore, it provides a structures framework for understanding and organizing information in within a database system, e.g. FIWARE stack. 

This example shows a workflow, how you can define or reuse use case specific data models and ensure FIWARE compatibility by merging these models with existing data model in FiLiP. The merged models can be used for interaction with FIWARE platform and in other information processing systems to establish interoperability.

## Use Case

In the following example use case, we will capture data and information of multiple PMUs in the FIWARE stack. Firstly, data models are created, then we will also show the process of exporting, sharing and reusing these data models. 

### Import dependencies

In [16]:
import os
os.chdir("/home/ubuntu/project/FiLiP")

from pydantic import ConfigDict, BaseModel
from pydantic.fields import Field, FieldInfo
from typing import Any, Optional
from pydantic_extra_types.coordinate import Coordinate
from datetime import datetime, date
import json 
from pprint import pprint

from filip.models import FiwareHeader
from filip.models.ngsi_v2.context import ContextEntityKeyValues
from filip.clients.ngsi_v2.cb import ContextBrokerClient



# Create data model

Good data modelling can harmonise representation formats and semantics for various applications. Therefore, we try to reuse existing data models. For our specific use case, we can also define our own data models.

## Common data model

Some data models are use case agnostic. For example, we reuse the data model of GeoCoordinates from [Schema.org](https://schema.org) to represent the location of PMUs.

In [17]:
class GeoCoordiantes(BaseModel):
    """
    https://schema.org/GeoCoordinates
    """
    model_config = ConfigDict(populate_by_name=True, coerce_numbers_to_str=True)  # Pydantic specific settings
    latitude: float = Field(
        alias="latitude",
        description="The latitude of a location. For example 37.42242 (WGS 84)."
    )
    longitude: float = Field(
        alias="longitude", 
        description="The longitude of a location. For example -122.08585 (WGS 84)."
    )
    address: Optional[str] = Field(
        alias="address",
        description="Physical address of the item.",
        default=None
    )
    addressCountry: Optional[str] = Field(
        alias="addressCountry",
        description="County code according to ISO 3166-1-alpha-2.",
        default=None
    )
    elevation: Optional[float] = Field(
        alias="elevation", 
        description="The elevation of a location (WGS 84). Values may be of the form 'NUMBER UNIT_OF_MEASUREMENT' (e.g., '1,000 m', '3,200 ft') while numbers alone should be assumed to be a value in meters.",
        default=None
    )
    postalCode: Optional[str] = Field(
        alias="postalCode", 
        description="The postal code. For example, 94043.",
        default=None
    )   

### Domain data models 

Some other data models already exist for specific domains. Sargon, for example, provides an open-source standardized semantic description for the electrical domain. 

In [18]:

class Angle(BaseModel):
    model_config = ConfigDict(coerce_numbers_to_str=True, extra="ignore")
    """
    https://sargon-n5geh.netlify.app/ontology/1.0/classes/angle/
    """
    angle: float = Field(
        default=100.0,
        description="Quantity, measures angle in degree."
    ) 
class Frequency(BaseModel):
    model_config = ConfigDict(coerce_numbers_to_str=True, extra="ignore")
    """
    https://sargon-n5geh.netlify.app/ontology/1.0/classes/Frequency
    """
    frequency: float = Field(
        default=50.0,
        description="Quantity, measures frequency in Hz."
    )
class Magnitude(BaseModel):
    model_config = ConfigDict(coerce_numbers_to_str=True, extra="ignore")
    """
    https://sargon-n5geh.netlify.app/ontology/1.0/classes/Maqnitute
    """
    magnitude: float = Field(
        default=50.0,
        description="Quantity, measures magnitude."
    )
class Rocf(BaseModel):
    model_config = ConfigDict(coerce_numbers_to_str=True, extra="ignore")
    """
    https://sargon-n5geh.netlify.app/ontology/1.0/classes/Rocf
    """
    rocf: float = Field(
        default=50.0,
        description="Quantity, measure rocof in df/dt."
    )
class Timestamp(BaseModel):
    model_config = ConfigDict(coerce_numbers_to_str=True, extra="ignore")
    """
    https://sargon-n5geh.netlify.app/ontology/1.0/classes/Timestamp
    """
    timestamp: float = Field(
        default=50.0,
        description="Quantity, time."
    )
     

In [19]:
class PMU(BaseModel):
    """
    https://sargon-n5geh.netlify.app/ontology/1.0/classes/PMU_
    """
    hasAngle: Any = Field(
        default=None,
        description="sargon:hasValue, link with a URI of ANgle",
    )
    hasFrequency: Any = Field(
        default=None,
        description="sargon:hasValue, link with a URI of Frequency",
    ) 
    hasMagnitude: Any = Field(
        default=None,
        description="sargon:hasValue, link with a URI of Magnitude",
    )
    hasRocf: Any = Field(
        default=None,
        description="sargon:hasValue, link with a URI of Rocf",
    )
    hasTimestamp: Any = Field(
        default=None,
        description="sargon:hasValue, link with a URI of Timestamp",
    )
    location: GeoCoordiantes
 
          


****Compliance with FIWARE****

After creating the data models, we also want to ensure the FIWARE compatibility. For this purpose, we can simply merge the data models of our use cases with the existing FIWARE models in FiLiP (>= 0.4.1)


In [20]:
class AngleFIWARE(Angle, ContextEntityKeyValues):
    type: str = FieldInfo.merge_field_infos(
    # Field info of the general FIWARE data model in FiLiP
    ContextEntityKeyValues.model_fields["type"],
    default="Angle"
)
class FrequencyFIWARE(Frequency, ContextEntityKeyValues):
    type: str = FieldInfo.merge_field_infos(
    # Field info of the general FIWARE data model in FiLiP
    ContextEntityKeyValues.model_fields["type"],
    default="Frequency"
)
class MagnitudeFIWARE(Magnitude, ContextEntityKeyValues):
    type: str = FieldInfo.merge_field_infos(
    # Field info of the general FIWARE data model in FiLiP
    ContextEntityKeyValues.model_fields["type"],
    default="Magnitude"
)
class RocfFIWARE(Rocf, ContextEntityKeyValues):
    type: str = FieldInfo.merge_field_infos(
    # Field info of the general FIWARE data model in FiLiP
    ContextEntityKeyValues.model_fields["type"],
    default="Rocf"
)
class TimestampFIWARE(Timestamp, ContextEntityKeyValues):
    type: str = FieldInfo.merge_field_infos(
    # Field info of the general FIWARE data model in FiLiP
    ContextEntityKeyValues.model_fields["type"],
    default="Timestamp"
)

     
class PMUFIWARE(PMU, ContextEntityKeyValues):
    type: str = FieldInfo.merge_field_infos(
    # First position is the field info of the parent class
    ContextEntityKeyValues.model_fields["type"],
    # set the default value
    default="CustomModels:PMU",
    # overwrite the title in the json-schema if necessary
    title="Type of the PMU",
    # overwrite the description
    description="https://sargon-n5geh.netlify.app/ontologies/Sargon.ttl",
    # validate the default value if necessary
    validate_default=True,
    # freeze the field if necessary
    frozen=True,
    # for more options see the pydantic documentation
    )

### Use the data model for your own application

Give the address of your context broker and change the environment variables SERVICE and SERVICE_PATH according to your settings in the entirety platform. 

In [21]:
CB_URL = "http://137.226.248.184:1026"
SERVICE = 'Test'
SERVICE_PATH = '/'
fiware_header = FiwareHeader(service=SERVICE,
                             service_path=SERVICE_PATH)
cb_client = ContextBrokerClient(url=CB_URL,
                                fiware_header=fiware_header)

Now we can export both the use case model and the FIWARE specific models to json-schema files and share it with other stakeholders that need the data.

In [22]:
use_case_model = PMU.model_json_schema()
pprint(use_case_model)
use_case_model = PMUFIWARE.model_json_schema()
pprint(use_case_model)

{'$defs': {'GeoCoordiantes': {'description': 'https://schema.org/GeoCoordinates',
                              'properties': {'address': {'anyOf': [{'type': 'string'},
                                                                   {'type': 'null'}],
                                                         'default': None,
                                                         'description': 'Physical '
                                                                        'address '
                                                                        'of '
                                                                        'the '
                                                                        'item.',
                                                         'title': 'Address'},
                                             'addressCountry': {'anyOf': [{'type': 'string'},
                                                                          {'type': 'null'}],
 

Initialize entities.

In [23]:
entity_list = cb_client.get_entity_list()

pmu1 = PMUFIWARE(
    id="myPMU1",
    type="PMU",
    location=GeoCoordiantes(latitude=40.5, longitude=50.3)
)
angle = AngleFIWARE(
    id="myAngle",
    type="angle",
    angle=120,
)
frequency = FrequencyFIWARE(
    id="myFrequency",
    type="frequency",
    voltage=210,
)
magnitude = MagnitudeFIWARE(
    id="myMagnitude",
    type="magnitude",
    rocof=120,
)
rocf = RocfFIWARE(
    id="myRocf", 
    type="rocof",
)

timestamp = TimestampFIWARE(
    id="myTimestamp",
    type="timestamp",
)

Post entities, update the data and add the data to the PMU entity.

In [26]:
cb_client.post_entity(entity=pmu1, key_values=True, update=True)
cb_client.post_entity(entity=angle, key_values=True, update=True)
cb_client.post_entity(entity=frequency, key_values=True, update=True)
cb_client.post_entity(entity=magnitude, key_values=True, update=True)



pprint(cb_client.get_entity_list())

pmu1.hasAngle = angle.id
pmu1.hasFrequency = frequency.id
pmu1.hasMagnitude = magnitude.id
angle.angle = 25.3
cb_client.update_entity_key_values(entity=angle)
cb_client.update_entity_key_values(entity=pmu1)

pmu_data = cb_client.get_entity(entity_id="myPMU1")

[ContextEntity(id='myPMU1', type='PMU', hasAngle=ContextAttribute(type='Text', value='myAngle', metadata={}), hasFrequency=ContextAttribute(type='Text', value='myFrequency', metadata={}), hasMagnitude=ContextAttribute(type='Text', value='myMagnitude', metadata={}), hasRocf=ContextAttribute(type='None', value=None, metadata={}), hasTimestamp=ContextAttribute(type='None', value=None, metadata={}), location=ContextAttribute(type='StructuredValue', value={'latitude': 40.5, 'longitude': 50.3, 'address': None, 'addressCountry': None, 'elevation': None, 'postalCode': None}, metadata={})),
 ContextEntity(id='myAngle', type='angle', angle=ContextAttribute(type='Number', value=25.5, metadata={})),
 ContextEntity(id='myFrequency', type='frequency', frequency=ContextAttribute(type='Number', value=50.0, metadata={}), voltage=ContextAttribute(type='Number', value=210.0, metadata={})),
 ContextEntity(id='myMagnitude', type='magnitude', magnitude=ContextAttribute(type='Number', value=50.0, metadata={}


### Export serialized data models

In order to share data models, we usually need to export the Pydantic classes to serialized data format, i.e. JSON Schema in this example.


In [ ]:
from pprint import pprint
import json
# Use case data model
weather_station_schema = PMU.model_json_schema()
with open("./pmu_schema.json", "w") as f:
    json.dump(weather_station_schema, f)
pprint(weather_station_schema)

# FIWARE specific data models
pmu_fiware_schema = PMUFIWARE.model_json_schema()
with open("./pmu_fiware_schema.json", "w") as f:
    json.dump(pmu_fiware_schema, f)
pprint(pmu_fiware_schema)

angle_fiware_schema = AngleFIWARE.model_json_schema()
with open("./angle_fiware_schema.json", "w") as f:
    json.dump(angle_fiware_schema, f)
pprint(angle_fiware_schema)

{'$defs': {'GeoCoordiantes': {'description': 'https://schema.org/GeoCoordinates',
                              'properties': {'address': {'anyOf': [{'type': 'string'},
                                                                   {'type': 'null'}],
                                                         'default': None,
                                                         'description': 'Physical '
                                                                        'address '
                                                                        'of '
                                                                        'the '
                                                                        'item.',
                                                         'title': 'Address'},
                                             'addressCountry': {'anyOf': [{'type': 'string'},
                                                                          {'type': 'null'}],
 


Use data models in other applications

As you can see above, the exported data model is based on normal JSON syntax, and is therefore both machine and human-readable. There are various libraries, that support parsing this data. For example, in Python jsonschema can be used to validate the data against the data model, and datamodel-code-generator can be used to generate python objects.


In [ ]:
# Validation
from jsonschema import validate
ws_data = cb_client.get_entity(entity_id="myPMU1", response_format="keyValues").model_dump()
validate(instance=ws_data, schema=pmu_fiware_schema)
print("The data conforms to the data model")

The data conforms to the data model


In [ ]:
# Parse serialized models
from pathlib import Path
from datamodel_code_generator import generate, DataModelType, InputFileType
output_path=Path("./pmu.py")
class_name="GeneratedPMU"
generate(Path("./pmu_fiware_schema.json"),
        class_name=class_name,
        output=output_path,
        output_model_type=DataModelType.PydanticV2BaseModel,
        use_subclass_enum=True,
        input_file_type=InputFileType.JsonSchema
        )
print(f"Pyhton object '{class_name}' generated in '{output_path}'")

Pyhton object 'GeneratedPMU' generated in 'pmu.py'


In [ ]:
# Use generated python object
from pmu import GeneratedPMU
ws_data = cb_client.get_entity(entity_id="myPMU1", response_format="keyValues").model_dump()
ws_generated = GeneratedPMU.model_validate(ws_data)
print(ws_generated.model_dump_json(indent=2))
print(ws_generated.location.model_dump_json(indent=2))

{
  "id": "myPMU1",
  "type": "PMU",
  "hasAngle": "myAngle",
  "hasFrequency": "myFrequency",
  "hasMagnitude": "myMagnitude",
  "hasRocf": null,
  "hasTimestamp": null,
  "location": {
    "latitude": 40.5,
    "longitude": 50.3,
    "address": null,
    "addressCountry": null,
    "elevation": null,
    "postalCode": null
  }
}
{
  "latitude": 40.5,
  "longitude": 50.3,
  "address": null,
  "addressCountry": null,
  "elevation": null,
  "postalCode": null
}
